# Importing Dependencies

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("max_colwidth", None)
plt.style.use('ggplot')
import plotly.express as px
import re
import string

- **numpy**: package for numerical computations in Python.
- **seaborn**: data visualization library built on top of Matplotlib. 
- **matplotlib.pyplot**: is a comprehensive library for creating static, animated, and interactive visualizations in Python. The pyplot module provides a MATLAB-like interface for creating various types of plots, such as line plots, scatter plots, histograms
- **plotly.express**: is a powerful library for creating interactive plots and dashboards. plotly.express is a high-level interface for creating a variety of charts with minimal code. It provides an easy way to generate interactive visualizations for exploratory data analysis.
- **string**: provides a collection of string constants and classes useful for various string manipulations.
- *pd.set_option("max_coldwith")*: Setting the maximum column width to be unlimited.
- *plt.style_use('ggplot')*: Setting the style of Matplotlib plots to emulate the style of the popular "ggplot" package from R.

In [3]:
df = pd.read_csv("C:\\Users\\tissi\\SENHEISER.csv")
df

,ID,Review,Title,Rating,Date
0,John C.,"i have owned the momentum 2’s for many years and love them. but, after the battery started to fail and took over 8hrs to charge i decided to upgrade. here are my thoughts:sound - really good, but different to the 2’s. took some time to get used to but i really like the sound now.battery - amazing. 60 hours is huge and makes such a difference not having to charge so regularly.comfort - the 2’s were great but these are on another level. can wear for hours on end without getting hot or ever feeling uncomfortable.look - the 2’s looked amazing with the ivory colour. these look ok, but do not stand out. build quality is excellent. no creaking on mine.controls - i hate the touch controls and miss having a button to adjust the volume. if adjusting, then it is easy to activate a control. changing the volume is hit and miss. i have turned touch controls off as they are too unreliable. it would be good to switch just some of the controls off and just have volume, or have the ability to change via the app. maybe having less controls would help it work better.connectivity - multiple devices works fine. i’ve had no issues but it niggles me that when you connect, you get a lovely voice that tells you it has connected, but when you connect a bluetooth device, a lady from essex tells you it is connected. it is so wierd they use different voices for different announcements.all in all a great pair of headphones but i wish sennheiser would address some minor niggles.",awesome headphones…. but…,4.0,08 10 2023
1,R Griffiths,"at first i felt these headphones were a mixed bag of tricks. i had read other reviewers had problems but having an earlier sennheiser model and being familiar with their app i hoped for the best. however it took ages to get going. the on off control is tricky - to get a bluetooth connection the on button must be pressed till the lights flash rapidly. it's also hard to tell if it's really switched off. i then had a nightmare trying to connect to the sennheiser app. my older set connects to both the app and bluetooth in seconds and works brilliantly but the momentum 4 took me the best part of an hour plus another thirty minutes to install a firmware update. the problem i had, i think, was i initially tried connecting using a fire10 tablet and the app just didn't like it. as a last attempt i tried the fire max 11 which worked at once. i then then downloaded the update by which time i had had enough electronics for one very stressful evening. the next morning i tried again with the fire 10 and all was well. it seems the update resolved issues connecting to the app but unless you have another device or can find another way to update it's frustrating. sennheiser do say their app is designed for phones primarily and do offer support with tablet related problems but in the event i didn't need to contact them. since that first attempt i have had absolutely no problems with connection at all on any of the three devices i use.the sennheiser sound is superb on the momentum 4. i very much like the sound check system which helps to establish your personal presets by offering several sets of three settings - you select your preferred out of each set. i like a wide range of music including jazz, opera, rock, metal and in a few minutes was able to save settings for each genre. i had done this before with my older sennheiser with great success but the sound on this newer set is brilliant. unlike other reviewers i like the app. once you find your way round it it's easy to use and the online manual is comprehensive and clearly written.there is only one button on the momentum 4 much to my relief - and as i have said that one needs a bit of learning to operate effectively. other controls are by touch and these work extremely well - swipe up and down for volume, touch for stop start etc. unless you are committed to buttons this is a great improvement. the set is extremely comfortable with a neat flat appearance

In [4]:
# Check for missing values in the "Review" column
missing_reviews = df[df['Review'].isna()]

# Print the number of missing reviews and their details
if not missing_reviews.empty:
    print("Missing Reviews:")
    print(missing_reviews)
else:
    print("No missing reviews found.")

No missing reviews found.


No missing reviews found, we won't deal with N/A.

Let's examine whether there are any repetitions in customer names. This consideration stems from the fact that occasionally individuals make purchases and leave reviews without creating a formal account, opting to enter as 'guests'.

In [11]:
def find_repeated_names_and_counts(df):
    repeated_names_series = df[df['ID'].duplicated(keep=False)]['ID']
    repeated_names_counts = repeated_names_series.value_counts()
    return repeated_names_counts

repeated_names_counts = find_repeated_names_and_counts(df)

print(f"Repeated names and their counts under 'ID':\n{repeated_names_counts}")


Repeated names and their counts under 'ID':
ID
Amazon Customer    7
AK                 3
Name: count, dtype: int64


It is evident that labels 'Amazon Customer' and 'AK' appears multiple times under the 'ID' column. To avoid having any future complication in the analysis let's find if those are repeated review of the same customer or different ones.

In [12]:
# Function to check if reviews for repeated names are the same or not
def check_reviews_similarity(df, column_name, review_column_name):
    repeated_names_df = df[df[column_name].duplicated(keep=False)]
    repeated_names_reviews = repeated_names_df.groupby(column_name)[review_column_name].unique()

    for name, reviews in repeated_names_reviews.items():
        if len(set(reviews)) == 1:
            print(f"The reviews for '{name}' are the same.")
        else:
            print(f"The reviews for '{name}' are different.")

column_to_check = 'ID'
review_column = 'Review'

check_reviews_similarity(df, column_to_check, review_column)

The reviews for 'AK' are different.
The reviews for 'Amazon Customer' are different.


Perfect just the names are the same

In [13]:
# Function to rename equals names
def rename_occurrences(df, column_name, names_to_rename):
    for name in names_to_rename:
        name_indices = df.index[df[column_name] == name].tolist()
        rename_mapping = {index: f'{name}_{i + 1}' for i, index in enumerate(name_indices)}
        df[column_name] = df.index.map(lambda x: rename_mapping.get(x, df.at[x, column_name]))

column_to_check = 'ID'
names_to_rename = ['AK', 'Amazon Customer']

# Rename equals names with unique names
rename_occurrences(df, column_to_check, names_to_rename)

This verification is crucial for sentiment analysis. The recurrence of the same name might be interpretated as a singular review from the same customer by the interpreter.
I experienced this situation. Specifically, this problem originated from the analysis of Vader's scores (which we will explore in the next section). In some instances, reviews exhibited identical scores despite variations in both the rating and sentiment. After the rename the scores changed according to the sentiment of the review.

In [7]:
df['Title'] = df['Title'].str.lower()
df['Review'] = df['Review'].str.lower()

Converting text to lowercase (using str.lower()) is a common preprocessing step in Natural Language Processing (NLP).

Text data can be **inconsistent** in terms of capitalization. By converting all text to lowercase, we ensure that the same word in different cases (e.g., "apple" and "Apple") is treated as the same word. This helps in standardizing the text and avoiding duplication in the analysis.

Lowercasing reduces the dimensionality of the data and makes it easier to handle. It simplifies the text and can help to mitigate the impact of case differences on downstream NLP tasks, using lowercase words as features can improve model performance by reducing sparsity.

While searching or matching text, converting everything to lowercase ensures that we won't miss matches due to differences in case. For example, when searching for "apple," you don't want to miss instances of "Apple."

Common words like "the," "and," "is," etc., are removed as they don't provide meaningful information. By converting text to lowercase, you ensure that stop words are consistently identified and removed.

Many NLP models, including pre-trained language models like BERT, are trained on lowercase text. Using an input(a dataframe in this case) not totally lowercased won't align with the language model's expections and will affect the performance.

In [8]:
def rating_into_sentiment(row):
    if row == 3.0:
        return 'Neutral'
    elif row >= 4.0:
        return 'Positive'
    elif row <= 2.0:
        return 'Negative'

# Convert the 'Rating' column to a numeric data type
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

df['Sentiment'] = df['Rating'].apply(rating_into_sentiment)

df

,ID,Review,Title,Rating,Date,Sentiment
0,John C.,"i have owned the momentum 2’s for many years and love them. but, after the battery started to fail and took over 8hrs to charge i decided to upgrade. here are my thoughts:sound - really good, but different to the 2’s. took some time to get used to but i really like the sound now.battery - amazing. 60 hours is huge and makes such a difference not having to charge so regularly.comfort - the 2’s were great but these are on another level. can wear for hours on end without getting hot or ever feeling uncomfortable.look - the 2’s looked amazing with the ivory colour. these look ok, but do not stand out. build quality is excellent. no creaking on mine.controls - i hate the touch controls and miss having a button to adjust the volume. if adjusting, then it is easy to activate a control. changing the volume is hit and miss. i have turned touch controls off as they are too unreliable. it would be good to switch just some of the controls off and just have volume, or have the ability to change via the app. maybe having less controls would help it work better.connectivity - multiple devices works fine. i’ve had no issues but it niggles me that when you connect, you get a lovely voice that tells you it has connected, but when you connect a bluetooth device, a lady from essex tells you it is connected. it is so wierd they use different voices for different announcements.all in all a great pair of headphones but i wish sennheiser would address some minor niggles.",awesome headphones…. but…,4.0,08 10 2023,Positive
1,R Griffiths,"at first i felt these headphones were a mixed bag of tricks. i had read other reviewers had problems but having an earlier sennheiser model and being familiar with their app i hoped for the best. however it took ages to get going. the on off control is tricky - to get a bluetooth connection the on button must be pressed till the lights flash rapidly. it's also hard to tell if it's really switched off. i then had a nightmare trying to connect to the sennheiser app. my older set connects to both the app and bluetooth in seconds and works brilliantly but the momentum 4 took me the best part of an hour plus another thirty minutes to install a firmware update. the problem i had, i think, was i initially tried connecting using a fire10 tablet and the app just didn't like it. as a last attempt i tried the fire max 11 which worked at once. i then then downloaded the update by which time i had had enough electronics for one very stressful evening. the next morning i tried again with the fire 10 and all was well. it seems the update resolved issues connecting to the app but unless you have another device or can find another way to update it's frustrating. sennheiser do say their app is designed for phones primarily and do offer support with tablet related problems but in the event i didn't need to contact them. since that first attempt i have had absolutely no problems with connection at all on any of the three devices i use.the sennheiser sound is superb on the momentum 4. i very much like the sound check system which helps to establish your personal presets by offering several sets of three settings - you select your preferred out of each set. i like a wide range of music including jazz, opera, rock, metal and in a few minutes was able to save settings for each genre. i had done this before with my older sennheiser with great success but the sound on this newer set is brilliant. unlike other reviewers i like the app. once you find your way round it it's easy to use and the online manual is comprehensive and clearly written.there is only one button on the momentum 4 much to my relief - and as i have said that one needs a bit of learning to operate effectively. other controls are by touch and these work extremely well - swipe up and down for volume, touch for stop start etc. unless you are committed to buttons this is a great improvement. the set is extremely comfortable with a n

This function is designed to convert star ratings into sentiment labels.

**rating_into_sentiment** takes a numeric value as input (the star rating) and returns the corresponding sentiment label.

It applies the rating_into_sentiment function to each element in the 'Rating' column and creates a new column called 'Sentiment' to store the corresponding sentiment labels.

In [9]:
def polish(Review):
    Review = re.sub('\W+', ' ', Review)                                   # Remove special characters
    Review = re.sub(r'[:;=]\s*[-]?[)D(\[\]/\\OpP]', '', Review)           # Remove emoticons
    Review = re.sub(r'[^\w\s:]', '', Review)                              # Remove punctuation
    Review = re.sub('\[.*?\]', '', Review)                                # Remove text inside square brackets
    Review = re.sub('https?://\S+|www\.\S+', '', Review)                  # Remove URLs
    Review = re.sub('<.*?>+', '', Review)                                 # Remove HTML tags
    Review = re.sub('[%s]' % re.escape(string.punctuation), '', Review)   # Remove remaining punctuation
    Review = re.sub('\n', '', Review)                                     # Remove newline characters
    Review = re.sub('\w*\d\w*', '', Review)                               # Remove words containing digits
    Review = re.sub('\d+', '', Review)                                    # Remove numbers
    return Review
    

def clean_urls(Review):
    return re.sub(r'http\S+', '', Review)

# Expansion of contractions
def contraction_expansion(Review):
    Review = re.sub(r"won\'t", "would not", Review)
    Review = re.sub(r"can\'t", "can not", Review)
    Review = re.sub(r"don\'t", "do not", Review)
    Review = re.sub(r"shouldn\'t", "should not", Review)
    Review = re.sub(r"wouldn\'t", "would not", Review)
    Review = re.sub(r"needn\'t", "need not", Review)
    Review = re.sub(r"hasn\'t", "has not", Review)
    Review = re.sub(r"haven\'t", "have not", Review)
    Review = re.sub(r"wasn\'t", "was not", Review)
    Review = re.sub(r"weren\'t", "were not", Review)
    Review = re.sub(r"mightn\'t", "might not", Review)
    Review = re.sub(r"didn\'t", "did not", Review)
    Review = re.sub(r"isn\'t", "is not", Review)
    Review = re.sub(r"aren\'t", "are not", Review)
    Review = re.sub(r"doesn\'t", "does not", Review)
    
    return Review

def data_cleaning(Review):
    Review = clean_urls(Review)
    Review = contraction_expansion(Review)
    Review = polish(Review)   
    return Review

**polish(Review)**: This function takes a text input Review and performs the following operations:

- Removes special characters using regular expressions and replaces them with a space.
- Removes emoticons by using a regular expression that matches common emoticons and replaces them with an empty string.
- Removes punctuation by using a regular expression that keeps only words and spaces.
- Removes text inside square brackets.
- Removes URLs.
- Removes HTML tags.
- Removes remaining punctuation.
- Removes newline characters.
- Removes words containing digits.
- Remove numbers

**clean_urls(Review)**: This function takes a text input Review and removes URLs (web links) from the text.

**contraction_expansion(Review)**: This function performs the expansion of contractions. It takes a text input Review and expands common contractions into their full forms using a series of regular expressions. For example, "won't" is expanded to "would not," "can't" to "can not," and so on. This is done to standardize contractions in text data.

**data_cleaning(Review)**: This function combines the three preprocessing functions mentioned above (clean_urls, contraction_expansion, and polish) to perform a comprehensive cleaning and preparation of the input text. It first removes URLs, then expands contractions, and finally, polishes the text by removing special characters, emoticons, and punctuation.
Leaving a *clean* review.

Let's try to put togheter the title togheter with  the review. The title is a more frank opinion on the product typically composed by few words

In [10]:
df['Full Review'] = df['Title'] + ': ' + df['Review']

In [11]:
df['Full Review'] = df['Full Review'].apply(data_cleaning)

In [12]:
stop_words = [
    'i', 'me', 'would', 'and', 'also', 'get', 'let', 'one', 'shall', 'should', 'may', 'might', 'could', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y'
]

len(stop_words)

149

In [13]:
df['Full Review'] = df['Full Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [14]:
df = df[['ID', 'Full Review', 'Review', 'Title', 'Rating', 'Sentiment', 'Date']]
df.head(1)

,ID,Full Review,Review,Title,Rating,Sentiment,Date
0,John C.,awesome headphones owned momentum many years love battery started fail took charge decided upgrade thoughts sound really good different took time used really like sound battery amazing hours huge makes difference not charge regularly comfort great another level wear hours end without getting hot ever feeling uncomfortable look looked amazing ivory colour look ok not stand build quality excellent no creaking mine controls hate touch controls miss button adjust volume adjusting easy activate control changing volume hit miss turned touch controls unreliable good switch controls volume ability change via app maybe less controls help work better connectivity multiple devices works fine no issues niggles connect lovely voice tells connected connect bluetooth device lady essex tells connected wierd use different voices different announcements great pair headphones wish sennheiser address minor niggles,"i have owned the momentum 2’s for many years and love them. but, after the battery started to fail and took over 8hrs to charge i decided to upgrade. here are my thoughts:sound - really good, but different to the 2’s. took some time to get used to but i really like the sound now.battery - amazing. 60 hours is huge and makes such a difference not having to charge so regularly.comfort - the 2’s were great but these are on another level. can wear for hours on end without getting hot or ever feeling uncomfortable.look - the 2’s looked amazing with the ivory colour. these look ok, but do not stand out. build quality is excellent. no creaking on mine.controls - i hate the touch controls and miss having a button to adjust the volume. if adjusting, then it is easy to activate a control. changing the volume is hit and miss. i have turned touch controls off as they are too unreliable. it would be good to switch just some of the controls off and just have volume, or have the ability to change via the app. maybe having less controls would help it work better.connectivity - multiple devices works fine. i’ve had no issues but it niggles me that when you connect, you get a lovely voice that tells you it has connected, but when you connect a bluetooth device, a lady from essex tells you it is connected. it is so wierd they use different voices for different announcements.all in all a great pair of headphones but i wish sennheiser would address some minor niggles.",awesome headphones…. but…,4.0,Positive,08 10 2023


In [15]:
df['Date'] = pd.to_datetime(df['Date'], format='%d %m %Y')

# Extract day, month, and year into separate columns
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

Transform the original date information into a more structured format and adds separate columns for the year, month, and day components, making it easier to analyze or filter data based on these temporal attributes.

In [16]:
# Save the DataFrame to a CSV file
df.to_csv('SENHEISER.csv', index=False)